# IMPORTS

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"  # specify which GPU(s) to be used
import numpy as np
import h5py
import matplotlib.pyplot as plt
import os.path
import tensorflow as tf 
import numpy as np


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# DATA LOADING

In [3]:
CTW_labelled = "/content/drive/MyDrive/CTW2020/Unzipped/CTW2020_labelled_data"
CTW_unlabelled="/content/drive/MyDrive/CTW2020/Unzipped/CTW2020_unlabelled_data"

In [4]:
def get_data(data_file):
    
    f = h5py.File(data_file, 'r')
    H_Re = f['H_Re'][:] #shape (sample size, 56, 924, 5)
    H_Im = f['H_Im'][:] #shape (sample size, 56, 924, 5)
    SNR = f['SNR'][:] #shape (sample size, 56, 5)
    Pos = f['Pos'][:] #shape(sample size, 3)
    f.close()
            
    return H_Re, H_Im, SNR, Pos        

In [5]:
### example code to load data from file_1.hdf4  ###

data_file = CTW_labelled+"/file_"+str(2)+".hdf5"
H_Re, H_Im, SNR, Pos = get_data(data_file)
print("H_Re is of shape {}".format(H_Re.shape))
print("H_Im is of shape {}".format(H_Im.shape))
print("SNR is of shape {}".format(SNR.shape))
print("Pos is of shape {}".format(Pos.shape))

H_Re is of shape (512, 56, 924, 5)
H_Im is of shape (512, 56, 924, 5)
SNR is of shape (512, 56, 5)
Pos is of shape (512, 3)


In [6]:

H_Re_T=tf.convert_to_tensor(H_Re)
H_Im_T=tf.convert_to_tensor(H_Im)
SNR_T=tf.convert_to_tensor(SNR)
Pos_T=tf.convert_to_tensor(Pos)


# Preprocessing 

A] Median Filter 

In [7]:
#amplitude created
Amp=tf.math.sqrt(tf.math.add(tf.math.square(H_Re_T),tf.math.square(H_Im_T)))



In [8]:
Amp[:,1,:,1].shape # (number of sample,number of subcarriers)

TensorShape([512, 924])

In [9]:
CSI1=Amp[:,1,:,0] #CSI Image of each location , 1st measurement 
CSI2=Amp[:,1,:,1] #CSI Image of each location , 2st measurement 
CSI3=Amp[:,1,:,2] #CSI Image of each location , 3st measurement 
CSI4=Amp[:,1,:,3] #CSI Image of each location , 4st measurement 
CSI5=Amp[:,1,:,4] #CSI Image of each location , 5st measurement 

In [10]:
CSI1

<tf.Tensor: shape=(512, 924), dtype=float16, numpy=
array([[0.10767, 0.149  , 0.179  , ..., 0.0803 , 0.0683 , 0.05087],
       [0.02602, 0.03723, 0.04733, ..., 0.04886, 0.04248, 0.0318 ],
       [0.05402, 0.0744 , 0.08887, ..., 0.02646, 0.0238 , 0.01753],
       ...,
       [0.4797 , 0.6553 , 0.786  , ..., 0.4478 , 0.3828 , 0.286  ],
       [0.01546, 0.02376, 0.03262, ..., 0.4336 , 0.3677 , 0.2732 ],
       [0.00798, 0.01151, 0.01384, ..., 0.02791, 0.02208, 0.0155 ]],
      dtype=float16)>

In [11]:
!pip install -q --no-deps tensorflow-addons

In [12]:
#Source:https://gist.github.com/bhawkins/3535131
#median filter algorithm on coulumns:1D

def medfilt (x, k):
    """Apply a length-k median filter to a 1D array x.
    Boundaries are extended by repeating endpoints.
    """
    assert k % 2 == 1, "Median filter length must be odd."
    #assert x.ndim == 1, "Input must be one-dimensional."
    k2 = (k - 1) // 2
    y = np.zeros ((len (x), k), dtype=x.dtype)
    y[:,k2] = x
    for i in range (k2):
        j = k2 - i
        y[j:,i] = x[:-j]
        y[:j,i] = x[0]
        y[:-j,-(i+1)] = x[j:]
        y[-j:,-(i+1)] = x[-1]
    return np.median (y, axis=1)



In [13]:
#test case:1D array

x=np.array([2,100,4,6])
medfilt(x,3)

array([2., 4., 6., 6.])

In [14]:
#test case for 2D image 1 D filter as explained in the paper
img=[[2,3,4],[100,2,3],[4,70000,3],[6,2,50000]]
imgnp=np.array(img)


In [15]:
imgnp

array([[    2,     3,     4],
       [  100,     2,     3],
       [    4, 70000,     3],
       [    6,     2, 50000]])

In [18]:
#test case to find median column wise median filter for sample image 
final0=np.zeros([1,4])
final=np.zeros([1,4])
for i in range(3):
  imgl=imgnp[:,i]
  final0=np.append(final0,final)
  final=medfilt(imgl,3)
final0=np.append(final0,final)
fin=np.transpose(final0.reshape(5,4))
fin1=np.delete(fin, [0,1], 1)

In [19]:
fin1

array([[2.e+00, 3.e+00, 4.e+00],
       [4.e+00, 3.e+00, 3.e+00],
       [6.e+00, 2.e+00, 3.e+00],
       [6.e+00, 2.e+00, 5.e+04]])

In [20]:
#median filter on actual CSI images (we have 5 csi images for, 5 measurements)

CSI1n=CSI1.numpy()

final1=np.zeros([1,512])
final2=np.zeros([1,512])

for i in range(924):
  img1=CSI1n[:,i]
  final2=np.append(final2,final1)
  final1=medfilt(img1,3)
final2=np.append(final2,final1)
CSI1fin=np.transpose(final2.reshape(926,512)) 
CSI1_fil=np.delete(CSI1fin, [0,1], 1)

CSI2n=CSI2.numpy()
final3=np.zeros([1,512])
final4=np.zeros([1,512])
for i in range(924):
  img2=CSI2n[:,i]
  final4=np.append(final4,final3)
  final3=medfilt(img2,3)
final4=np.append(final4,final3)
CSI1fin=np.transpose(final4.reshape(926,512)) 
CSI2_fil=np.delete(CSI1fin, [0,1], 1)

CSI3n=CSI3.numpy()
CSI1_fil3=np.delete(CSI1fin, [0,1], 1)
final5=np.zeros([1,512])
final6=np.zeros([1,512])
for i in range(924):
  img3=CSI3n[:,i]
  final6=np.append(final6,final5)
  final5=medfilt(img3,3)
final6=np.append(final6,final5)
CSI1fin=np.transpose(final6.reshape(926,512)) 
CSI3_fil=np.delete(CSI1fin, [0,1], 1)

CSI4n=CSI4.numpy()
final7=np.zeros([1,512])
final8=np.zeros([1,512])
for i in range(924):
  img4=CSI4n[:,i]
  final8=np.append(final8,final7)
  final7=medfilt(img4,3)
final8=np.append(final8,final7)
CSI4_fin=np.transpose(final8.reshape(926,512))


CSI5n=CSI5.numpy()
final10=np.zeros([1,512])
final9=np.zeros([1,512])
for i in range(924):
  img5=CSI5n[:,i]
  final9=np.append(final10,final9)
  final10=medfilt(img5,3)
final10=np.append(final10,final9)
CSI1fin=np.transpose(final10.reshape(926,512)) 
CSI5_fil=np.delete(CSI1fin, [0,1], 1)



In [21]:
CSI1_fil.shape

(512, 924)

In [22]:
CSI1_fil

array([[0.10766602, 0.14904785, 0.17895508, ..., 0.08032227, 0.06829834,
        0.0508728 ],
       [0.05401611, 0.07440186, 0.08886719, ..., 0.04885864, 0.04248047,
        0.03179932],
       [0.04147339, 0.05838013, 0.06982422, ..., 0.02645874, 0.02380371,
        0.01753235],
       ...,
       [0.06140137, 0.07995605, 0.09295654, ..., 0.43359375, 0.36767578,
        0.27319336],
       [0.01545715, 0.02375793, 0.03262329, ..., 0.43359375, 0.36767578,
        0.27319336],
       [0.00798035, 0.01151276, 0.01383972, ..., 0.02790833, 0.02207947,
        0.01550293]])

In [23]:
CSI2_fil.shape

(512, 924)

In [24]:
CSI2_fil

array([[0.10644531, 0.14855957, 0.1796875 , ..., 0.08068848, 0.06854248,
        0.05026245],
       [0.04360962, 0.0612793 , 0.07531738, ..., 0.04016113, 0.03025818,
        0.01893616],
       [0.04025269, 0.05429077, 0.06341553, ..., 0.01953125, 0.01430511,
        0.01048279],
       ...,
       [0.01722717, 0.02331543, 0.02812195, ..., 0.0958252 , 0.07989502,
        0.05718994],
       [0.01065063, 0.01669312, 0.02113342, ..., 0.02757263, 0.02322388,
        0.0171051 ],
       [0.01065063, 0.01669312, 0.02113342, ..., 0.02757263, 0.02322388,
        0.0171051 ]])

B]Normalisation 

NameError: ignored

# Pretext Model 

# Downstream model  